### This file carries out orthogonal projection from a Cartesian point in space to an ellipsoid Earth.

Equatorial Radius $r_{e} =  6,378,136.6 $ $ (m)$

Polar Radius $r_{p} =   6,356,751.9 $ $ (m)$


Mathematical Expression of Earth in Cartesian coordinates:

$$ \frac{x^2}{{r_e}^2} + \frac{y^2}{{r_p}^2} + \frac{z^2}{{r_e}^2} = 1 $$


In [1]:
import numpy as np
import math 
import matplotlib as plt

import utilities as ut

class Earth():
    def __init__(self):
        
        '''
        Properties of the Earth and its position.
        '''

        self.re = 6378136.6
        self.rp = 6356751.9
        self.omega = 7.2921159 * 10**(-5)  
        self.mass = 5.9722*10**24


    def ellipse_equation(self, x, y, z):
        return (x^2)/(self.re^2) + (y^2)/(self.rp^2) + (z^2)/(self.re^2) - 1 

    def distane_to_surface(self, state, tolerance = 1e-6, max_iterations = 1000):
        '''
        Finds the minimum distance between the specified point and the ellipse
        using Newton's method.

        NEEDS TO BE OPTIMISED FOR POLAR COORDINATES
        '''

        '''
        x comes in as a set of 3D polar coordinates, since we want it to be in cartesian 

        x1 - ellipse coordinate
        x2 - satellite coordinates
        ''' 

        #We translate into the plane which is relevant to us
        x = np.asarray(state)
        relevant_state = np.array([[x[0]], [x[1]]])
        x2 = ut.p_to_c(relevant_state)

        t = math.atan2(x2[1], x2[0])

        a = self.re
        b = self.rp
            
        iterations = 0
        error = tolerance
        errors = []
        ts = []
                
        while error >= tolerance and iterations < max_iterations:
            cost = math.cos(t)
            sint = math.sin(t)
            x1 = np.array([a * cost, b * sint])
            xp = np.array([-a * sint, b * cost])
            xpp = np.array([-a * cost, -b * sint])
            delta = x1 - x2
            dp = np.dot(xp, delta)
            dpp = np.dot(xpp, delta) + np.dot(xp, xp)

            t -= dp / dpp
            error = abs(dp / dpp)
            errors.append(error)
            ts.append(t)
            iterations += 1
        
        ts = np.array(ts)
        errors = np.array(errors)
        y = np.linalg.norm(x1 - x2)
        success = error < tolerance and iterations < max_iterations
        return dict(x = t, y = y, error = error, iterations = iterations, success = success, xs = ts,  errors = errors)

    def air_density(self, distance):
        '''
        Atmosphere code, gets you the pressure.
        '''

        R = 8.3145
        M_air = 28.96 * 10 ** -3
        T = self.get_Temperature(distance)
        g = self.get_g(distance)
        H = R*T / (M_air * g)
        
        rho = (6 * 10 ** -10) * np.exp(-(distance - 175000)/H)
        return rho

    def get_Temperature(self, distance):
        return 24
    
    def get_g(self, distance):
        return 9.8
